<a href="https://colab.research.google.com/github/shahnupur1901/Filtrone-App/blob/master/DeepLearningModels/ModelForTopwearArticleTypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paramaggarwal/fashion-product-images-small
! unzip /content/fashion-product-images-small.zip

In [ ]:
import numpy as np
import pandas as pd
import os
path = "/content/myntradataset"
csv_path = "/content/myntradataset/styles.csv"
img_path = "/content/myntradataset/images"
df = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=False)

df = df[df['baseColour'].notna()]
df['id'] = df['id'].apply(lambda x: str(x)+'.jpg')
df = df[df['id'].isin(os.listdir(img_path))]
df = df.reset_index(drop=True)
df = df.groupby(['articleType']).filter(lambda x: len(x) >= 20)
print('DF shape:', df.shape)

DF shape: (44005, 10)


In [ ]:
a = df['articleType'].unique()
print(a)
new_df = df[(df['articleType'] == "Shirts") | (df['articleType'] == "Sweatshirts") | (df['articleType'] == "Tshirts") | (df['articleType'] == "Tops") | (df['articleType'] == "Shirts") | (df['articleType'] == "Dresses") | (df['articleType'] == "Kurtis") | (df['articleType'] == "Sweaters") ]

['Shirts' 'Jeans' 'Watches' 'Track Pants' 'Tshirts' 'Socks' 'Casual Shoes'
 'Belts' 'Flip Flops' 'Handbags' 'Tops' 'Bra' 'Sandals' 'Shoe Accessories'
 'Sweatshirts' 'Deodorant' 'Formal Shoes' 'Bracelet' 'Lipstick' 'Flats'
 'Kurtas' 'Sports Shoes' 'Shorts' 'Briefs' 'Sarees'
 'Perfume and Body Mist' 'Heels' 'Sunglasses' 'Innerwear Vests' 'Pendant'
 'Nail Polish' 'Laptop Bag' 'Scarves' 'Dresses' 'Night suits' 'Skirts'
 'Wallets' 'Ring' 'Kurta Sets' 'Clutches' 'Backpacks' 'Caps' 'Trousers'
 'Earrings' 'Camisoles' 'Boxers' 'Jewellery Set' 'Dupatta' 'Capris'
 'Lip Gloss' 'Bath Robe' 'Mufflers' 'Tunics' 'Jackets' 'Trunk'
 'Lounge Pants' 'Face Wash and Cleanser' 'Necklace and Chains'
 'Duffel Bag' 'Sports Sandals' 'Foundation and Primer' 'Sweaters'
 'Free Gifts' 'Tracksuits' 'Fragrance Gift Set' 'Bangle' 'Nightdress'
 'Ties' 'Leggings' 'Highlighter and Blush' 'Kurtis' 'Mobile Pouch'
 'Messenger Bag' 'Face Moisturisers' 'Compact' 'Accessory Gift Set'
 'Kajal and Eyeliner' 'Suspenders' 'Lip Line

In [ ]:
new_df.shape[0]
new_df.groupby(['articleType']).size()

articleType
Dresses         464
Kurtis          234
Shirts         3215
Sweaters        277
Sweatshirts     285
Tops           1762
Tshirts        7066
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(new_df,test_size = 0.1, random_state=42)
train_df, validation_df = train_test_split(new_df,test_size = 0.2, random_state=42)

In [ ]:
df.head()
df.sample(frac=1,axis=1)
batch_size = 16

from keras_preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    #validation_split=0.2
)
validation_image_generator = ImageDataGenerator(
    rescale= 1./255,
    #validation_split=0.2
)

training_generator = train_image_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=img_path,
    x_col="id",
    y_col="articleType",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="training"
)

validation_generator = validation_image_generator.flow_from_dataframe(
    dataframe=validation_df,
    directory=img_path,
    x_col="id",
    y_col="articleType",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="validation"
)
test_generator = validation_image_generator.flow_from_dataframe(
    dataframe=test_df,
    directory=img_path,
    x_col="id",
    y_col="articleType",
    target_size=(96,96),
    batch_size=batch_size,
    #subset="validation"
)

classes = len(training_generator.class_indices)
print(training_generator.class_indices)

Found 10642 validated image filenames belonging to 7 classes.
Found 2661 validated image filenames belonging to 7 classes.
Found 1331 validated image filenames belonging to 7 classes.
{'Dresses': 0, 'Kurtis': 1, 'Shirts': 2, 'Sweaters': 3, 'Sweatshirts': 4, 'Tops': 5, 'Tshirts': 6}


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization,Dropout
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = (96,96,3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(classes, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()
batch_size = 64
total_train = train_df.shape[0]
total_validation = validation_df.shape[0]
vgg_classifier = model.fit(training_generator,
steps_per_epoch=(total_train//batch_size),
epochs = 20,
validation_data=validation_generator,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose=1)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 45, 45, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 22, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0

In [ ]:
model.save('modelForTopArticleType85.h5')
result = model.evaluate(test_generator,batch_size=32)
print("test_loss, test accuracy",result)

84/84 [==============================] - 5s 55ms/step - loss: 0.4479 - accuracy: 0.8370
test_loss, test accuracy [0.4479343593120575, 0.8369646668434143]


In [ ]:
import json
with open('modelfortopweararticletype.json', 'w') as fp:
    json.dump(training_generator.class_indices, fp)